In [1]:
### STEP 0: Importing packages ###

import csv
import pandas as pd

import os
import pymysql
import mysql.connector
from sqlalchemy import create_engine

import matplotlib.pyplot as plt

In [2]:
### STEP 1: Ingesting a local file ###

# in this step, the local file will be taken in (in the form of a csv, in this case), and we will use Pandas
# to create a datafram that can then be manipulated in Python
df = pd.read_csv(r'cereal.csv')
df

name mfr type  calories  protein  fat  sodium  fiber  \
0                   100% Bran   N    C        70        4    1     130   10.0   
1           100% Natural Bran   Q    C       120        3    5      15    2.0   
2                    All-Bran   K    C        70        4    1     260    9.0   
3   All-Bran with Extra Fiber   K    C        50        4    0     140   14.0   
4              Almond Delight   R    C       110        2    2     200    1.0   
..                        ...  ..  ...       ...      ...  ...     ...    ...   
72                    Triples   G    C       110        2    1     250    0.0   
73                       Trix   G    C       110        1    1     140    0.0   
74                 Wheat Chex   R    C       100        3    1     230    3.0   
75                   Wheaties   G    C       100        3    1     200    3.0   
76        Wheaties Honey Gold   G    C       110        2    1     200    1.0   

    carbo  sugars  potass  vitamins  shelf  weight  cups     rating  
0     5.0       6     280        25      3     1.0  0.33  68.402973  
1     8.0       8     135         0      3     1.0  1.00  33.983679  
2     7.0       5     320        25      3     1.0  0.33  59.425505  
3     8.0       0     330        25      3     1.0  0.50  93.704912  
4    14.0       8      -1        25      3     1.0  0.75  34.384843  
..    ...     ...     ...       ...    ...     ...   ...        ...  
72   21.0       3      60        25      3     1.0  0.75  39.106174  
73   13.0      12      25        25      2     1.0  1.00  27.753301  
74   17.0       3     115        25      1     1.0  0.67  49.787445  
75   17.0       3     110        25      1     1.0  1.00  51.592193  
76   16.0       8      60        25      1     1.0  0.75  36.187559  

[77 rows x 16 columns]

In [4]:
### STEP 2: A. sending file to JSON
# NB. this step includes Benchmark II -- error message will be produced if a file with the same 
#     name already exists in the local system!

# in this step, the existing Pandas DF that was generated in Step 1 will be routed to a JSON file that will
# then save to the same directory that is currently being used.
df.to_json(r'cereal_json.json', if_exists='fail')

In [5]:
### B. sending file to sqlite

# in this step, the existing Pandas DF that was generated in Step 1 will be routed to a sqlite database that 
# will then save to the same directory that is currently being used. Additionally, it will define the table 
# within that database, based on the name that I have given it (in this case 'cereal_data').
engine = create_engine('sqlite:///save_cereal.db', echo=True)
sqlite_connection = engine.connect()

sqlite_table = "cereal_data"
df.to_sql(sqlite_table, sqlite_connection, if_exists='fail')

2022-03-15 21:16:37,868 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2022-03-15 21:16:37,871 INFO sqlalchemy.engine.base.Engine ()
2022-03-15 21:16:37,872 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2022-03-15 21:16:37,888 INFO sqlalchemy.engine.base.Engine ()
2022-03-15 21:16:37,895 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("cereal_data")
2022-03-15 21:16:37,901 INFO sqlalchemy.engine.base.Engine ()


ValueError: Table 'cereal_data' already exists.

In [26]:
### STEP 3: Modifying the number of columns (adding a column) ###

# creating a copy of the original DF
df_new = df.copy()

# adding a new column for calories per carb within the new DF
df_new['calories_per_carb'] = df['calories']/df['carbo']
df_new

name mfr type  calories  protein  fat  sodium  fiber  \
0                   100% Bran   N    C        70        4    1     130   10.0   
1           100% Natural Bran   Q    C       120        3    5      15    2.0   
2                    All-Bran   K    C        70        4    1     260    9.0   
3   All-Bran with Extra Fiber   K    C        50        4    0     140   14.0   
4              Almond Delight   R    C       110        2    2     200    1.0   
..                        ...  ..  ...       ...      ...  ...     ...    ...   
72                    Triples   G    C       110        2    1     250    0.0   
73                       Trix   G    C       110        1    1     140    0.0   
74                 Wheat Chex   R    C       100        3    1     230    3.0   
75                   Wheaties   G    C       100        3    1     200    3.0   
76        Wheaties Honey Gold   G    C       110        2    1     200    1.0   

    carbo  sugars  potass  vitamins  shelf  weight  cups     rating  \
0     5.0       6     280        25      3     1.0  0.33  68.402973   
1     8.0       8     135         0      3     1.0  1.00  33.983679   
2     7.0       5     320        25      3     1.0  0.33  59.425505   
3     8.0       0     330        25      3     1.0  0.50  93.704912   
4    14.0       8      -1        25      3     1.0  0.75  34.384843   
..    ...     ...     ...       ...    ...     ...   ...        ...   
72   21.0       3      60        25      3     1.0  0.75  39.106174   
73   13.0      12      25        25      2     1.0  1.00  27.753301   
74   17.0       3     115        25      1     1.0  0.67  49.787445   
75   17.0       3     110        25      1     1.0  1.00  51.592193   
76   16.0       8      60        25      1     1.0  0.75  36.187559   

    calories_per_carb  
0           14.000000  
1           15.000000  
2           10.000000  
3            6.250000  
4            7.857143  
..                ...  
72           5.238095  
73           8.461538  
74           5.882353  
75           5.882353  
76           6.875000  

[77 rows x 17 columns]

In [44]:
### STEP 4: A. saving the new df as CSV (local file) ###
# NB. this step includes Benchmark II -- error message will be produced if a file with the same 
#     name already exists in the local system!

# in this step, the new DF that was created in Step 3 will be saved to a CSV file using Pandas.
df_new.to_csv('new_cereal.csv', if_exists='fail')

TypeError: to_csv() got an unexpected keyword argument 'if_exists'

In [29]:
### B. saving the new df as JSON ###

# in this step, the new Pandas DF that was generated in Step 3 will be routed to a JSON file that will
# then save to the same directory that is currently being used.
df_new.to_json(r'new_cereal_json.json', if_exists='fail')

In [33]:
### C. saving the new df to sqlite ###

# in this step, the new Pandas DF that was generated in Step 3 will be routed to a sqlite database that 
# will then save to the same directory that is currently being used. Additionally, it will define the table 
# within that database, based on the name that I have given it (in this case 'new_cereal_data').
engine = create_engine('sqlite:///save_new_cereal.db', echo=True)
sqlite_connection = engine.connect()

sqlite_table = "new_cereal_data"
df_new.to_sql(sqlite_table, sqlite_connection, if_exists='fail')

2022-03-16 17:59:30,258 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2022-03-16 17:59:30,264 INFO sqlalchemy.engine.base.Engine ()
2022-03-16 17:59:30,269 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2022-03-16 17:59:30,272 INFO sqlalchemy.engine.base.Engine ()
2022-03-16 17:59:30,298 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("new_cereal_data")
2022-03-16 17:59:30,302 INFO sqlalchemy.engine.base.Engine ()
2022-03-16 17:59:30,307 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("new_cereal_data")
2022-03-16 17:59:30,308 INFO sqlalchemy.engine.base.Engine ()
2022-03-16 17:59:30,312 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE new_cereal_data (
	"index" BIGINT, 
	name TEXT, 
	mfr TEXT, 
	type TEXT, 
	calories BIGINT, 
	protein BIGINT, 
	fat BIGINT, 
	sodium BIGINT, 
	fiber FLOAT, 
	carbo FLOAT, 
	sugars BIGINT, 
	potass BIGINT, 
	vitamins BIGINT, 
	shelf BIGINT,

In [39]:
### STEP 5: generating number of records and number of columns
    ### A. original DF
        # rows:
print("There are " + str(len(df)) + " rows in the original DataFrame.")

There are 77 rows in the original DataFrame.


In [41]:
    ### A. original DF
        # columns:
print("There are " + str(len(df.columns)) + " columns in the original DataFrame.")

There are 16 columns in the original DataFrame.


In [42]:
    ### B. new DF
        # rows:
print("There are " + str(len(df_new)) + " rows in the new DataFrame.")

There are 77 rows in the new DataFrame.


In [43]:
    ### B. new DF
        # columns:
print("There are " + str(len(df_new.columns)) + " columns in the new DataFrame.")

There are 17 columns in the new DataFrame.
